In [ ]:
import sqlite3
import pandas as pd
from tqdm import tqdm
from nltk import sent_tokenize
import pickle
import regex as re

from gensim.models.word2vec import LineSentence
from gensim.models.phrases import Phrases, ENGLISH_CONNECTOR_WORDS

from time import perf_counter

import utils
from params import sql_db, comments_file, annotated_submissions_file

## Setup

In [ ]:
# Define location and name of SQL database, create a connection object
conn = sqlite3.connect(sql_db)

# Converting the submissions to manually annotate them
submissions_file = './data/submissions.csv'

## Manually filter submissions
- I only select submissions with at least 25 comments (according to the submission metadata, not the actual amount of comments I have in the database.
- Based on the submission titles, I select submissions which are about a single movie and not 'meta-comments' (e.g., "why isn't there an official discussion for X?")

In [ ]:
submissions = pd.read_sql('SELECT * FROM submissions', conn)

print(len(submissions))

In [ ]:
submissions.to_csv(submissions_file, index=False)

In [ ]:
# I created a new column which is empty for official discussions (num_comments >= 25 only)
all_submissions = pd.read_csv(annotated_submissions_file, sep=';')
all_submissions.dropna(subset=['created'], inplace=True)

# if editing film titles, also do
#all_submissions.drop(columns="film_title", inplace=True)

In [ ]:
all_submissions = all_submissions.astype({'score': 'int', 'num_comments': 'int', 'created':'int'})
all_submissions.loc[:,'date'] = pd.to_datetime(all_submissions.loc[:,'created'], unit='s')

In [ ]:
submissions = all_submissions.loc[(all_submissions['num_comments'] >= 25) & (pd.isna(all_submissions['discussion_thread']))]

In [ ]:
print(len(submissions))

In [ ]:
titles = submissions.title.tolist()

to_remove = [
    "[Spoilers]",
    "[SPOILERS]",
    "[SPOILERS}",
    "(SPOILERS)",
    "[SPOLERS]",
    "[spoilers]",
    
    "(un-Official Discussion)",
    "Un-Official Discussion :",
    "Un-Official Discussion:",
    "Psuedo-Official Discussion:",
    
    "discussion (Spoilers inside)",
    "Official International Release Discussion:",
    "Official International Discussion -",
    "Official International Release Discussion Thread -",
    
    "- Official Discussion Thread",
    "Official Discussion Thread:",
    
    "Discussion Thread.",
    "Official Discussion:",
    "Official Discussion -",
    "Official Discussion-",
    "Discussion Thread",
    
    "IMAX re-release",
    "(International Thread)",
    ", Netflix",
    "(UK release)",
    "(US Release)",
    "(International Release)",
    "(UK Release)",
    "(US Thread)",
    "(Thread Vol. 2)",
    "Official Discussion #2 -",
    "Movie Discussion",
    
    "Small Axe:",
    "4K Restoration",
    "The Saturday Official",
    "Official Late-Comer Megathread -",
    "(Roadshow 70mm Cut)",
    "(Theatrical Cut)",
    "The Pre-Official",
    "discussion thread",
    "OFFICIAL DISCUSSION",
    "Official Discussion",
    "[Serious Replies Only]",
    "(Early Release)",
    "(Spoilers inside)",
    "(Wide Release)",
    "(Netflix Release)",
    "(theater release)",
    "(2nd Thread)",
    "SPOILERS"
]

remove_other = [
    "-Official 48 fps Discussion thread.",
    r" \.",
    r"\d+/\d+  -",
    "31 Days of Halloween -",
    r"(July|August) \d{1,2}",
    '"',
    "'",
    "Official",
    r"- \d+/\d+",
    "\(Un\)"
]

replace_other = [
    # typos
    ["Godzilla: King of the Monstars", "Godzilla: King of the Monsters"],
    ["Thor: Rangarok", "Thor: Ragnarok"],
    # for tmdb matching
    ["The Hunger Games: Mockingjay - Part II", "The Hunger Games: Mockingjay - Part 2"],
    ["The Hunger Games: Mockingjay Part 2", "The Hunger Games: Mockingjay - Part 2"],
    ["The Hunger Games: Mockingjay Part 1", "The Hunger Games: Mockingjay - Part 1"],
    ["The Edge of Tomorrow", "Edge of Tomorrow"],
    ["T2 Trainspotting: Judgement Day", "T2 Trainspotting"],
    ["Jack Reacher: Never Stop Never Stopping", "Jack Reacher: Never Go Back"],
    ["The Girl in the Spiders Web: A New Dragon Tattoo Story", "The Girl in the Spiders Web"],
    ["T2 Trainspotting: Judgement Day", "T2 Trainspotting"],
    ["T2 Trainspotting: Battle Across Time", "T2 Trainspotting"],
    ["John Wick 2", "John Wick: Chapter 2"],
    ["Oceans 8", "Ocean's Eight"],
    ["Your Name", "君の名は。"],
    ["Sailor Moon Eternal", "劇場版 美少女戦士セーラームーンEternal 前編"],
    ["The Christmas Chronicles 2", "The Christmas Chronicles: Part Two"],
    ["The Croods 2", "The Croods: A New Age"],
    ["El Camino", "El Camino: A Breaking Bad Movie"]
    
]

film_titles = []
for title in titles:
    for j in to_remove:
        title = title.replace(j, "")
        
    for j in remove_other:
        title = re.sub(j, "", title)
        
    for i, j in replace_other:
        if i == title.strip():
            title = title.replace(i, j)
        
    title = title.strip()
        
    title = title.strip()    
    film_titles.append(title)

In [ ]:
for i in film_titles:
    if re.search('君の名は。', i):
        print(i)

In [ ]:
submissions['film_title'] = film_titles

In [ ]:
submissions.loc[submissions.film_title.str.contains("Camino")]

In [ ]:
df = pd.merge(all_submissions, submissions, how="outer")

In [ ]:
df.to_csv(annotated_submissions_file, sep=';', index=False)

## Select matching comments

In [ ]:
x = "'"+"','".join(submissions['submission_id'].tolist())+"'"

In [ ]:
start = perf_counter()
comments = pd.read_sql(f"SELECT * FROM comments WHERE submission_id IN ({x})", conn)
print(len(comments))

print(perf_counter()-start)

In [ ]:
comments.to_csv(comments_file, sep=';', index=False)

In [ ]:
comments = pd.read_csv(comments_file, sep=';')

In [ ]:
comments.head()